In [11]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
pd.set_option('display.max_colwidth',1000)

In [12]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import codecs
#data = pd.read_csv('PreprocessedData.csv', header = None, encoding='utf-8', engine='python') ## for older one 
data = pd.read_csv('FinalListPreprocessedData.csv', header = None, encoding='utf-8', engine='python')
data.columns = ['ID','User','CREATED_AT','LABEL','stemming_words']
data = data.iloc[1: , :] # start from first column
data.head()

,ID,User,CREATED_AT,LABEL,stemming_words
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොයා බැලීමේ වැඩ පිලිවලක් ඕනේ
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා කරමින් හැකි ඉක්මණින් ආර්ථිකය
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුරෝපීය රට වලට විසා නිකුත් නොකි
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුර
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට


In [13]:
data.loc[data['LABEL']=='Crime',"LABEL"] = 0
data.loc[data['LABEL']=='Entertainment',"LABEL"] = 1
data.loc[data['LABEL']=='Politics',"LABEL"] = 3
data.loc[data['LABEL']=='Sport',"LABEL"] = 4
data.loc[data['LABEL']=='Other',"LABEL"] = 5

In [14]:
data.head()

,ID,User,CREATED_AT,LABEL,stemming_words
1,2.0,1.42E+18,7/29/2021 2:40,5,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොයා බැලීමේ වැඩ පිලිවලක් ඕනේ
2,3.0,1.42E+18,7/27/2021 10:40,3,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා කරමින් හැකි ඉක්මණින් ආර්ථිකය
3,4.0,1.42E+18,7/25/2021 20:56,5,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුරෝපීය රට වලට විසා නිකුත් නොකි
4,5.0,1.42E+18,7/25/2021 20:53,5,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුර
5,6.0,1.42E+18,7/25/2021 20:52,5,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට


In [15]:
data['LABEL'].value_counts()

4    86
3    76
5    69
0    25
1    14
Name: LABEL, dtype: int64

In [16]:
############### Apply tfidf vectorizer ######################

In [17]:
data_x=data["stemming_words"]
data_y=data["LABEL"]

cv = TfidfVectorizer()

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=4)

In [19]:
x_traincv=cv.fit_transform(x_train)

In [20]:
a = x_traincv.toarray()
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
len(a[0])

651

In [22]:
###########  MultiNomial Naive bayes ############

In [23]:
mnb = MultinomialNB()
y_train = y_train.astype('int')
mnb.fit(x_traincv,y_train)

MultinomialNB()

In [24]:
x_testcv=cv.transform(x_test)
prediction = mnb.predict(x_testcv)
prediction

array([4, 3, 4, 4, 3, 4, 5, 4, 4, 5, 4, 4, 5, 3, 3, 4, 4, 4, 5, 3, 3, 4,
       4, 4, 3, 4, 3, 3, 3, 3, 4, 0, 4, 4, 3, 4, 5, 5, 3, 3, 3, 4, 4, 5,
       3, 4, 3, 5, 5, 5, 4, 5, 4, 4])

In [25]:
actual = np.array(y_test)
actual

array([4, 3, 4, 4, 4, 4, 5, 4, 4, 3, 4, 3, 0, 4, 0, 5, 0, 5, 5, 3, 3, 1,
       4, 4, 3, 1, 3, 0, 3, 3, 4, 0, 5, 3, 3, 4, 0, 3, 3, 3, 5, 4, 4, 1,
       5, 4, 3, 5, 5, 3, 5, 4, 4, 4], dtype=object)

In [26]:
count = 0
for i in range (len(prediction)):
    if prediction[i] == actual[i]:
        count = count +1

In [27]:
count #correct prediction

32

In [28]:
len(prediction) #total prediction

54

In [30]:
32/54 ###### probbility

0.5925925925925926